# 巡回セールスマン問題(TSP)をPyQUBOを用いて解く

In [15]:
%matplotlib inline
from pyqubo import Array, Placeholder, Spin, solve_qubo, Constraint, OneHotEncInteger
import neal
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time

In [16]:
def plot_city(cities, city_order=None):
    n_city = len(cities)
    cities_dict = dict(cities)
    G = nx.Graph()
    for city in cities_dict:
        G.add_node(city)
        
    # draw path
    for i in range(n_city):
        city_index1 = city_order[i]
        city_index2 = city_order[(i+1) % n_city]
        G.add_edge(cities[city_index1][0], cities[city_index2][0])

    plt.figure(figsize=(3,3))
    pos = nx.spring_layout(G)
    nx.draw_networkx(G, cities_dict)
    plt.axis("off")
    plt.show()

#都市u,v間の距離を出力
def dist(i, j, cities):
    pos_i = cities[i][1]
    pos_j = cities[j][1]

    return np.sqrt((pos_i[0] - pos_j[0])**2 + (pos_i[1] - pos_j[1])**2)

In [17]:
# 都市の名前と座標のデータを用意 list[("name", (x, y))]
cities = [
    ("a", (0, 0)),
    ("b", (1, 3)),
    ("c", (0, 4)),
    ("d", (2, 1)),
    ("e", (4, 0))
]
#plot_city(cities,{})

In [18]:
#都市の数
n_city = len(cities)
x = Array.create('c', (n_city, n_city), 'BINARY') # x[j,v]：都市vに時刻j

In [19]:
# セールスマンは各都市に１度だけ訪問する制約
time_const = 0.0
for v in range(n_city):
    time_const += Constraint((sum(x[j, v] for j in range(n_city)) - 1)**2, label="time{}".format(v))
    
# ある時刻に1つの都市しか存在しない制約
city_const = 0.0
for j in range(n_city):
    city_const += Constraint((sum(x[j, v] for v in range(n_city)) - 1)**2, label="city{}".format(j))

In [20]:
# 経路の総距離を記述
distance = 0.0
for u in range(n_city):
    for v in range(n_city):
        for j in range(n_city):
            # 時刻jに都市u, 時刻j+1に都市vにいた場合の都市i,j間の距離
            d_uv = dist(u, v, cities)
            distance += d_uv * x[j, u] * x[(j+1)%n_city, v]
            #print((j+1)%n_city)

In [21]:
# ハミルトニアンを構築
A = Placeholder("A")
B = Placeholder("B")
H = distance + A * (time_const + city_const)

In [22]:
# モデルをコンパイル
model = H.compile()

In [26]:
# QUBOを作成
feed_dict = {'A': 10.0}

In [27]:
bqm = model.to_bqm(feed_dict=feed_dict)

In [28]:
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=200, num_sweeps=200)

# Decode solution
decoded_samples = model.decode_sampleset(sampleset, feed_dict=feed_dict)
best_sample = min(decoded_samples, key=lambda x: x.energy)
num_broken = len(best_sample.constraints(only_broken=True))
print("number of broken constarint = {}".format(num_broken))

number of broken constarint = 0


In [29]:
best_sample.constraints(only_broken=True)

{}

In [30]:
best_sample.sample.items()

dict_items([('c[0][0]', 1), ('c[3][4]', 0), ('c[1][4]', 1), ('c[2][4]', 0), ('c[0][2]', 0), ('c[1][0]', 0), ('c[1][1]', 0), ('c[1][2]', 0), ('c[0][4]', 0), ('c[0][3]', 0), ('c[2][3]', 1), ('c[3][0]', 0), ('c[2][1]', 0), ('c[2][2]', 0), ('c[3][1]', 1), ('c[3][2]', 0), ('c[3][3]', 0), ('c[4][0]', 0), ('c[1][3]', 0), ('c[0][1]', 0), ('c[4][1]', 0), ('c[2][0]', 0), ('c[4][2]', 1), ('c[4][3]', 0), ('c[4][4]', 0)])